# Preparation of Dataset from Esaki et al, 2019 Mol Inform

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

DATAPATH = "../data"

In [12]:
from rdkit.Chem import SDMolSupplier
from rdkit import Chem

path = os.path.join(DATAPATH, "original", "esaki2019.sdf")
suppl = SDMolSupplier(path)
R = []

cols = ["ChEMBL ID", "Compound name", "Dataset", "Observed", "Predicted"]
for i, mol in tqdm(enumerate(suppl)):
    if mol is None: continue
    props = mol.GetPropsAsDict()
    r = [props[c] for c in cols] + [Chem.MolToSmiles(mol)]
    R += [r]
df= pd.DataFrame(R, columns = cols+["smiles"])

5278it [00:05, 1046.17it/s]


In [22]:
clf = []
for o in df["Observed"].tolist():
    if o == "Stable":
        c = 1
    else:
        c = 0
    clf.append(c)
df["obs"]=clf

In [23]:
df.head()

,ChEMBL ID,Compound name,Dataset,Observed,Predicted,smiles,clf,obs
0,CHEMBL503643,,Test,Stable,Stable,CCOC(=O)c1cc2cc(C(=O)O)ccc2[nH]1,1,1
1,CHEMBL501315,,Training,Unstable,Unstable,COc1cccc(CCN(Cc2ccccc2-c2ccc(CN3C[C@H](C)N[C@H...,0,0
2,CHEMBL1163186,,Training,Stable,Stable,CCN(C(=O)NCc1ccc(S(C)(=O)=O)cc1)C1CCN(CCC(c2cc...,1,1
3,CHEMBL1084953,,Test,Stable,Stable,CN(C)CCCC(C)(C)c1ccc(C(C)(C)C)cc1,1,1
4,CHEMBL1085225,,Training,Moderate,Moderate,Cc1cc(C#N)nc(C)c1C(=O)N1CCC(C)(N2CCC(N3C(=O)N(...,0,0


In [30]:
df.rename(columns={"ChEMBL ID": "ChEMBL_ID"}, inplace=True)

In [32]:
#check invalid smiles

def invalid_mols(data, smi_col):
    smiles = data[smi_col].tolist()
    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    pre = len(data)
    for i, mol in enumerate(mols):
        if mol is None:
            smi=smiles[i]
            data.drop(index = i, inplace = True)
    post= len(data)
    print(str(pre-post))
    return data

df = invalid_mols(df, "smiles")

0


In [34]:
df[["ChEMBL_ID", "smiles", "obs"]].to_csv(os.path.join(DATAPATH, "processed", "esaki2021.csv"), index=False)